In [1]:
import numpy as np

def eigen_analysis(matrix):
    eigenvalues, eigenvectors = np.linalg.eig(matrix)
    
    # A⋅v=λ⋅v
    for i in range(len(eigenvalues)):
        lambda_v = eigenvalues[i] * eigenvectors[:, i]
        A_v = np.dot(matrix, eigenvectors[:, i])
        if not np.allclose(lambda_v, A_v):
            print(f"Check failed for λ: {eigenvalues[i]} & v: {eigenvectors[:, i]}")
        else:
            print(f"Check successed for λ: {eigenvalues[i]} & v: {eigenvectors[:, i]}")
    
    return eigenvalues, eigenvectors

matrix = np.array([[4, 1], [2, 3]])
eigenvalues, eigenvectors = eigen_analysis(matrix)
print("Eigen values: ", eigenvalues)
print("Eigen vectors:\n", eigenvectors)


Check successed for λ: 5.0 & v: [0.70710678 0.70710678]
Check successed for λ: 2.0 & v: [-0.4472136   0.89442719]
Eigen values:  [5. 2.]
Eigen vectors:
 [[ 0.70710678 -0.4472136 ]
 [ 0.70710678  0.89442719]]
